In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## 分词器的获取
Unsurprisingly, there are many more techniques out there. To name a few:

Byte-level BPE, as used in GPT-2

WordPiece, as used in BERT

SentencePiece or Unigram, as used in several multilingual models

You should now have sufficient knowledge of how tokenizers work to get started with the API.

In [1]:
from transformers import AutoTokenizer

In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "I love her"
]
# 默认返回的是list格式，而不是tensor格式
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
print(tokenizer.decode(inputs['input_ids'][0]))
print(tokenizer.decode(inputs['input_ids'][1]))
print(tokenizer.decode(inputs['input_ids'][2]))

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  1045,  2293,  2014,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
[CLS] i've been waiting for a huggingface course my whole life. [SEP]
[CLS] i hate this so much! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] i love her [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


### 对于2个句子，tokenizer处理方法

In [11]:
inputs=tokenizer( "I've been waiting for a HuggingFace course my whole life.","I hate this so much!")
print(tokenizer.decode(inputs['input_ids']))


# sentence1->list sentence2->list
inputs=tokenizer( ["I've been waiting for a HuggingFace course my whole life.","I hate this so much!"],["I've been waiting for a HuggingFace course my whole life.","I hate this so much!"])

print(tokenizer.decode(inputs['input_ids'][0]))


[CLS] i've been waiting for a huggingface course my whole life. [SEP] i hate this so much! [SEP]
[CLS] i've been waiting for a huggingface course my whole life. [SEP] i've been waiting for a huggingface course my whole life. [SEP]


In [6]:
#切换另外一个tokenizer
bert_tokz=AutoTokenizer.from_pretrained("bert-base-uncased")
inputs=bert_tokz( "I've been waiting for a HuggingFace course my whole life.","I hate this so much!")
inputs
# token_type_ids表示是第一个句子，还是第二个句子

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 1045, 5223, 2023, 2061, 2172, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## 获得模型

In [7]:
from transformers import AutoModel

In [34]:
model=AutoModel.from_pretrained(checkpoint)
# model=AutoModel.from_pretrained(checkpoint,num_labels=33,ignore_mismatched_sizes=True)

outputs=model(**inputs)
print(outputs.last_hidden_state.shape)
model

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([3, 16, 768])


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [18]:
last_layer=model.get_submodule("transformer.layer.5.output_layer_norm")

## 用于分类的模型

In [19]:
from transformers import AutoModelForSequenceClassification

In [31]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
# outputs=model(**inputs)
# outputs = model(**inputs)
# print(outputs.logits.shape)

In [32]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [33]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

## 用于生成的的模型

In [6]:
from transformers import AutoModelForCausalLM

## Config类

In [16]:
from transformers import BertConfig, BertModel,AutoConfig
from fastbook import *
import fastbook

fastbook.setup_book()


In [30]:
# Building the config
# config = AutoConfig.from_pretrained("bert-base-cased",hidden_dropout_prob=0.3)
# 如果指定具体的类型，可以指定，这里覆盖了默认的参数
config = BertConfig.from_pretrained("bert-base-cased",hidden_dropout_prob=0.3)

# 有了config就可以创建具体的模型了
model = BertModel(config)
# 当然，还是使用与训练的更好
# model=BertModel.from_pretrained('bert-base-cased')


In [28]:
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [33]:
# 保存，读取与训练模型
model.save_pretrained('a/my')
AutoModel.from_pretrained('a/my')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.3, inplace=False)
  

## 数据集

In [35]:
from datasets import load_dataset
# glue是由10多种数据集构成的benckmark
#mrpc是数据集，比较2句话是否是一个意思
#  ['cola', 'sst2', 'mrpc', 'qqp', 'stsb', 'mnli', 'mnli_mismatched', 'mnli_matched', 'qnli', 'rte', 'wnli', 'ax']
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Found cached dataset glue (C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
#可以按行访问
raw_datasets['train'][0:1]
# 也可以按列访问
# raw_datasets['train']['sentence1']

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'],
 'label': [1],
 'idx': [0]}

In [40]:
raw_datasets.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [17]:
def tokenizr_func(example):
    # 把sentence1,sentence2连到一起进行 数值化处理，返回的格式是list
    return tokenizer(example['sentence1'],example['sentence2'],padding=True,truncation=True,max_length=128)

# 批量数值化原始数据集
ds=raw_datasets.map(tokenizr_func,batched=True)
# 移除多余的列， 重命名目标
ds=ds.remove_columns(['sentence1','sentence2','idx']).rename_column('label','labels')
ds

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

### 分割数据集

In [13]:
raw_datasets['train'].train_test_split(0.25)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2751
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 917
    })
})

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

## 动态与静态padding

In [170]:
from torch.utils.data import dataset, sampler, dataloader
import torch

def padding_fix(example,maxlen=512):
    ret= tokenizer(example['sentence1'],example['sentence2'],padding=True,truncation=True,max_length=maxlen)
    return ret
def padding_dynamic(example,maxlen=512):
    ret= tokenizer(example['sentence1'],example['sentence2'])
    return ret

In [166]:
ds_train=raw_datasets['train'].map(padding_fix,batched=True)
ds_train=ds_train.remove_columns(['sentence1','sentence2','idx']).rename_column('label','labels')

# dataset.map返回的数据集，sample是存放在list中的，加上这句话，把list转换成对应的tensor
ds_train=ds_train.with_format('torch')

#可以看到，如果使用padding_fix，所有的样本都会被padding成最长的
dls=dataloader.DataLoader(ds_train,batch_size=5,shuffle=False)
for k,t in enumerate(dls):
    print(t['input_ids'].shape,t['attention_mask'].shape,t['labels'].shape)
    if k==5:
        break

torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])
torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])
torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])
torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])
torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])
torch.Size([5, 89]) torch.Size([5, 89]) torch.Size([5])


In [171]:
ds_train=raw_datasets['train'].map(padding_dynamic,batched=True)
ds_train=ds_train.remove_columns(['sentence1','sentence2','idx']).rename_column('label','labels')
# dataset.map返回的数据集，sample是存放在list中的，加上这句话，把list转换成对应的tensor
ds_train=ds_train.with_format('torch')

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
dls=dataloader.DataLoader(ds_train,batch_size=5,shuffle=False,collate_fn=data_collator)

#可以看到，如果使用padding_dynamic，每个batch会被 padding成batch中最长的长度
for k,t in enumerate(dls):
    print(t['input_ids'].shape,t['attention_mask'].shape,t['labels'].shape)
    if k==5:
        break

  0%|          | 0/4 [00:00<?, ?ba/s]

torch.Size([5, 67]) torch.Size([5, 67]) torch.Size([5])
torch.Size([5, 62]) torch.Size([5, 62]) torch.Size([5])
torch.Size([5, 61]) torch.Size([5, 61]) torch.Size([5])
torch.Size([5, 79]) torch.Size([5, 79]) torch.Size([5])
torch.Size([5, 64]) torch.Size([5, 64]) torch.Size([5])
torch.Size([5, 68]) torch.Size([5, 68]) torch.Size([5])


In [180]:
# data_collator接受一个map，都没个把每个value(list)都转换成对于的tensor,如果value的元素长度参差不齐时，使用padding补齐后，在转成tensor
batch=data_collator(ds_train[20:31])
print(batch['labels'].shape,batch['attention_mask'].shape)

torch.Size([11]) torch.Size([11, 68])


In [159]:
kk=33
# tokenizer.convert_ids_to_tokens(ds_train['input_ids'][kk]),len(ds_train['input_ids'][kk])
# torch.tensor(ds_train['input_ids'][22:55]).shape
# ds_train['attention_mask'][22:25]
len(ds_train['input_ids']),ds_train['input_ids'][100].shape
ds_train['labels']

tensor([1, 0, 1,  ..., 1, 1, 0])

### 从pandas获得dataset

In [7]:
from datasets import Dataset,DatasetDict
import pandas
df=pandas.read_csv('a.csv')

ds = Dataset.from_pandas(df)

## 训练

In [59]:
from transformers import Trainer,TrainingArguments,DataCollatorWithPadding
import numpy as np

In [54]:
def custom_token_fn(example):
    return tokenizer(example['sentence1'],example['sentence2'],truncation=True)
token_ds=raw_datasets.map(custom_token_fn,batched=True)
token_ds=token_ds.remove_columns(['sentence1','sentence2','idx']).rename_column('label','labels')
token_ds=token_ds.with_format('torch')

Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-4f16cc51848abc4a.arrow
Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-5e14ebdb0725d9fa.arrow
Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-988406d9e4008365.arrow


In [64]:
# evaluation_strategy 在测试集上测试 的时间间隔策略
#learning_rate
#per_device_train_batch_size
#per_device_eval_batch_size
#num_train_epochs
#weight_decay

# warmup_ratio:  ??
# lr_scheduler_type: ??

args=TrainingArguments('output',evaluation_strategy='epoch')
def compute_metrics(data):
    logits,labels=data
    
    pred_labels=np.argmax(logits,-1)
    acc=(pred_labels==labels).mean()
    
    return {"acc":acc}

def compute_metrics_mrpc(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #这是evaluate返回函数的固定方法compute,根据预测与标注，返回dict（包含所有指标）
    return metric.compute(predictions=predictions, references=labels)

model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english\snapshots\3d65bad49c7ba6f71920504507a8927f4b9db6c0\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 51

In [65]:
trainer=Trainer(model,args,
                train_dataset=token_ds['train'],
                eval_dataset=token_ds['validation'],
                data_collator=DataCollatorWithPadding(tokenizer),
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
               )

In [66]:
trainer.train()

***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Epoch,Training Loss,Validation Loss,Acc
1,No log,0.402058,0.838235
2,0.507300,0.483808,0.840686
3,0.295000,0.722676,0.840686


***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to output\checkpoint-500
Configuration saved in output\checkpoint-500\config.json
Model weights saved in output\checkpoint-500\pytorch_model.bin
tokenizer config file saved in output\checkpoint-500\tokenizer_config.json
Special tokens file saved in output\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to output\checkpoint-1000
Configuration saved in output\checkpoint-1000\config.json
Model weights saved in output\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-1000\tokenizer_config.json
Special tokens file saved in output\checkpoint-1000\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.3287302403668173, metrics={'train_runtime': 625.4079, 'train_samples_per_second': 17.595, 'train_steps_per_second': 2.202, 'total_flos': 204499651379856.0, 'train_loss': 0.3287302403668173, 'epoch': 3.0})

In [73]:
### test dataset
test_predict=trainer.predict(token_ds['test'])
test_predict.predictions.shape,test_predict.label_ids.shape,test_predict.metrics

***** Running Prediction *****
  Num examples = 1725
  Batch size = 8


((1725, 2),
 (1725,),
 {'test_loss': 0.8396176099777222,
  'test_acc': 0.8191304347826087,
  'test_runtime': 22.1269,
  'test_samples_per_second': 77.959,
  'test_steps_per_second': 9.762})

### 与pytorch 训练

In [159]:
import torch
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

In [160]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
#数据集准备
def custom_token_fn(example):
    return tokenizer(example['sentence1'],example['sentence2'],truncation=True)
token_ds=raw_datasets.map(custom_token_fn,batched=True)
token_ds=token_ds.remove_columns(['sentence1','sentence2','idx']).rename_column('label','labels')
token_ds=token_ds.with_format('torch')

Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-aef651eabbc0fe31.arrow
Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-d98532cb24e169be.arrow
Loading cached processed dataset at C:/Users/Administrator/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-554ae6d17f8981ec.arrow


In [161]:
train_loader=DataLoader(token_ds['train'],batch_size=16,shuffle=True,collate_fn=DataCollatorWithPadding(tokenizer))
val_loader=DataLoader(token_ds['validation'],batch_size=16,shuffle=True,collate_fn=DataCollatorWithPadding(tokenizer))

In [175]:
# 定义模型

model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2).to(DEVICE)

loading configuration file config.json from cache at C:\Users\Administrator/.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english\snapshots\3d65bad49c7ba6f71920504507a8927f4b9db6c0\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.0",
  "vocab_size": 30522
}

loading weights 

In [176]:
@torch.no_grad()
def evaluate_accuracy(dls):
    accuracy=[]
    model.eval()
    for batch in dls:
        b={k:v.to(DEVICE) for k,v in batch.items()}
        logits=model(**b).logits
        pred_labels=logits.argmax(-1)
        accuracy.append((b['labels']==pred_labels).float().mean().item())
    model.train()
    return np.mean(accuracy)
evaluate_accuracy(val_loader)

0.39663461538461536

In [177]:
from tqdm.auto import tqdm
from transformers import get_scheduler



epoch=3
steps=epoch*len(train_loader)

progress=tqdm(range(steps))
# get_submodule('classifier')
optimizer=torch.optim.AdamW(model.get_submodule('classifier').parameters(),5e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=steps,
)

ce=torch.nn.functional.cross_entropy
model.train()
for _ in range(epoch):
    print(evaluate_accuracy(val_loader))
    for batch in train_loader:
        b={k:v.to(DEVICE) for k,v in batch.items()}
        logits=model(**b).logits
        loss=ce(logits,b['labels'])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress.update(1)

  0%|          | 0/690 [00:00<?, ?it/s]

0.4014423076923077
0.4110576923076923
0.4855769230769231


In [174]:
evaluate_accuracy(val_loader)

0.8317307692307693